# Unit 8b — SOLID Principles

**Purpose:** Master the five SOLID design principles that form the foundation of maintainable, flexible, and scalable object-oriented software design.

---

## 📚 Table of Contents

1. [Overview of SOLID](#1.-Overview-of-SOLID)
2. [Single Responsibility Principle (SRP)](#2.-Single-Responsibility-Principle-(SRP))
3. [Open/Closed Principle (OCP)](#3.-Open/Closed-Principle-(OCP))
4. [Liskov Substitution Principle (LSP)](#4.-Liskov-Substitution-Principle-(LSP))
5. [Interface Segregation Principle (ISP)](#5.-Interface-Segregation-Principle-(ISP))
6. [Dependency Inversion Principle (DIP)](#6.-Dependency-Inversion-Principle-(DIP))
7. [Exercises](#7.-Exercises)

---

## Prerequisites

Before starting this unit, you should be comfortable with:
- Unit 7 (OOP Foundations) — classes, objects, inheritance basics
- Unit 8a (Advanced OOP Techniques) — ABCs, type hints, properties

## 🎯 Learning Objectives

By the end of this unit, you will be able to:

**🏗️ Design Principles**
- Understand and apply all five SOLID principles
- Identify violations of SOLID in existing code
- Refactor code to follow SOLID principles

**📋 Single Responsibility**
- Design classes with a single, well-defined responsibility
- Recognize when a class is doing too much

**🔓 Open/Closed**
- Create extensible designs that don't require modifying existing code
- Use inheritance and composition to add new behavior

**🔄 Liskov Substitution**
- Ensure subclasses can substitute for their base classes without breaking behavior
- Avoid inheritance hierarchies that violate behavioral contracts

**📦 Interface Segregation**
- Design focused interfaces rather than large, general-purpose ones
- Use multiple small ABCs instead of one large ABC

**⬆️ Dependency Inversion**
- Depend on abstractions rather than concrete implementations
- Use dependency injection for flexible, testable code

In [ ]:
# Setup: Import ABC for use throughout this notebook
from abc import ABC, abstractmethod
import json

---

# 1. Overview of SOLID

**SOLID** is an acronym for five design principles that help create maintainable, flexible, and scalable object-oriented software:

| Letter | Principle | Key Idea |
|--------|-----------|----------|
| **S** | Single Responsibility | A class should have one reason to change |
| **O** | Open/Closed | Open for extension, closed for modification |
| **L** | Liskov Substitution | Subtypes must be substitutable for base types |
| **I** | Interface Segregation | Many specific interfaces over one general interface |
| **D** | Dependency Inversion | Depend on abstractions, not concretions |

These principles were introduced by Robert C. Martin ("Uncle Bob") and have become foundational to good object-oriented design.

The following sections demonstrate each principle with Python examples, showing both **violations** (❌ BAD) and **correct implementations** (✅ GOOD).

---

# 2. Single Responsibility Principle (SRP)

> "A class should have only one reason to change."

Each class should focus on a single responsibility or concern. When a class handles multiple unrelated tasks, changes to one task may inadvertently affect others.

## 2.1 Violation: One Class Doing Too Many Things

In [ ]:
# ❌ BAD: One class doing too many things
class UserManagerBad:
    """Violates SRP: handles data, validation, persistence, AND notifications."""
    
    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email
    
    def validate_email(self) -> bool:  # Validation logic
        return "@" in self.email
    
    def save_to_database(self) -> None:  # Database logic
        print(f"Saving {self.name} to database...")
    
    def send_welcome_email(self) -> None:  # Email logic
        print(f"Sending welcome email to {self.email}...")


# Problem: If email format changes, or database schema changes, or email template changes,
# we have to modify this same class — violating SRP
user = UserManagerBad("Alice", "alice@example.com")
if user.validate_email():
    user.save_to_database()
    user.send_welcome_email()

## 2.2 Correct Approach: Separate Responsibilities

In [ ]:
# ✅ GOOD: Each class has one responsibility

class User:
    """Data class — just holds user data."""
    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email


class EmailValidator:
    """Responsibility: Validates email addresses."""
    
    @staticmethod
    def is_valid(email: str) -> bool:
        return "@" in email and "." in email


class UserRepository:
    """Responsibility: Handles database operations for users."""
    
    def save(self, user: User) -> None:
        print(f"Saving {user.name} to database...")
    
    def find_by_email(self, email: str) -> User | None:
        print(f"Looking up user with email {email}...")
        return None  # Simulated lookup


class EmailService:
    """Responsibility: Handles sending emails."""
    
    def send_welcome(self, user: User) -> None:
        print(f"Sending welcome email to {user.email}...")
    
    def send_notification(self, user: User, message: str) -> None:
        print(f"Sending notification to {user.email}: {message}")


# Usage with SRP — each component can change independently
user = User("Alice", "alice@example.com")

if EmailValidator.is_valid(user.email):
    UserRepository().save(user)
    EmailService().send_welcome(user)

### Benefits of SRP

| Aspect | Before (Violation) | After (SRP) |
|--------|-------------------|-------------|
| **Testing** | Must test all concerns together | Test each class in isolation |
| **Changes** | Risky — may affect unrelated code | Safe — changes are isolated |
| **Reuse** | Difficult — tied to specific context | Easy — use components anywhere |
| **Understanding** | Complex class with many methods | Simple, focused classes |

---

# 3. Open/Closed Principle (OCP)

> "Software entities should be open for extension, but closed for modification."

You should be able to add new functionality by creating new classes, not by modifying existing, tested code.

## 3.1 Violation: Modifying Existing Code for New Features

In [ ]:
# ❌ BAD: Must modify existing code to add new discount types
class DiscountCalculatorBad:
    """Violates OCP: Every new discount type requires modifying this class."""
    
    def calculate(self, price: float, discount_type: str) -> float:
        if discount_type == "percentage":
            return price * 0.9
        elif discount_type == "fixed":
            return price - 10
        elif discount_type == "seasonal":  # Added later — had to modify class!
            return price * 0.8
        elif discount_type == "vip":  # Added later — had to modify again!
            return price * 0.75
        # Every new type requires modifying this class and re-testing everything
        else:
            return price


calc = DiscountCalculatorBad()
print(f"Percentage: ${calc.calculate(100, 'percentage')}")
print(f"Seasonal: ${calc.calculate(100, 'seasonal')}")

## 3.2 Correct Approach: Open for Extension

In [ ]:
# ✅ GOOD: Open for extension, closed for modification

class Discount(ABC):
    """Abstract base for discounts — new types extend this."""
    
    @abstractmethod
    def apply(self, price: float) -> float:
        pass


class PercentageDiscount(Discount):
    """Apply a percentage discount."""
    
    def __init__(self, percent: float):
        self.percent = percent
    
    def apply(self, price: float) -> float:
        return price * (1 - self.percent / 100)


class FixedDiscount(Discount):
    """Apply a fixed amount discount."""
    
    def __init__(self, amount: float):
        self.amount = amount
    
    def apply(self, price: float) -> float:
        return max(0, price - self.amount)


class SeasonalDiscount(Discount):
    """Seasonal multiplier discount — added WITHOUT modifying existing classes!"""
    
    def __init__(self, multiplier: float):
        self.multiplier = multiplier
    
    def apply(self, price: float) -> float:
        return price * self.multiplier


class DiscountCalculator:
    """Calculator that works with ANY discount — never needs modification."""
    
    def calculate(self, price: float, discount: Discount) -> float:
        return discount.apply(price)


# Usage — calculator never changes, just pass different discounts
calc = DiscountCalculator()
print(f"10% off $100: ${calc.calculate(100, PercentageDiscount(10))}")
print(f"$15 off $100: ${calc.calculate(100, FixedDiscount(15))}")
print(f"Seasonal 20% off: ${calc.calculate(100, SeasonalDiscount(0.8))}")

In [ ]:
# Adding new discount types is easy — just create a new class!

class BuyOneGetOneDiscount(Discount):
    """BOGO: Buy one, get second at reduced price."""
    
    def __init__(self, second_item_percent: float = 50):
        self.second_item_percent = second_item_percent
    
    def apply(self, price: float) -> float:
        # For simplicity, assume price is for 2 items
        return price * (1 - self.second_item_percent / 200)


# Works immediately with existing calculator — no modifications needed!
print(f"BOGO 50% off second: ${calc.calculate(100, BuyOneGetOneDiscount(50))}")

---

# 4. Liskov Substitution Principle (LSP)

> "Objects of a superclass should be replaceable with objects of its subclasses without breaking the program."

If class `B` extends class `A`, you should be able to use `B` anywhere `A` is expected without unexpected behavior.

## 4.1 Classic Violation: Square-Rectangle Problem

In [ ]:
# ❌ BAD: Square violates LSP — changing width doesn't behave like Rectangle

class RectangleBad:
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height
    
    def set_width(self, width: float) -> None:
        self.width = width
    
    def set_height(self, height: float) -> None:
        self.height = height
    
    def area(self) -> float:
        return self.width * self.height


class SquareBad(RectangleBad):
    """Violates LSP: changing one dimension changes both!"""
    
    def __init__(self, side: float):
        super().__init__(side, side)
    
    def set_width(self, width: float) -> None:
        self.width = width
        self.height = width  # Unexpected side effect!
    
    def set_height(self, height: float) -> None:
        self.width = height  # Unexpected side effect!
        self.height = height


def test_rectangle(rect: RectangleBad) -> None:
    """This function expects Rectangle behavior."""
    rect.set_width(5)
    rect.set_height(4)
    expected_area = 5 * 4  # 20
    actual_area = rect.area()
    print(f"Expected: {expected_area}, Actual: {actual_area}, OK: {expected_area == actual_area}")


# Rectangle works as expected
print("Testing Rectangle:")
test_rectangle(RectangleBad(1, 1))

# Square breaks the expectation — LSP violation!
print("Testing Square (violates LSP):")
test_rectangle(SquareBad(1))

## 4.2 Correct Approach: Separate Abstractions

In [ ]:
# ✅ GOOD: Use composition or separate abstractions

class Shape(ABC):
    """Abstract shape — all shapes have area."""
    
    @abstractmethod
    def area(self) -> float:
        pass


class Rectangle(Shape):
    """Rectangle with independent width and height."""
    
    def __init__(self, width: float, height: float):
        self._width = width
        self._height = height
    
    @property
    def width(self) -> float:
        return self._width
    
    @property
    def height(self) -> float:
        return self._height
    
    def area(self) -> float:
        return self._width * self._height


class Square(Shape):
    """Square is its own shape, not a modified Rectangle."""
    
    def __init__(self, side: float):
        self._side = side
    
    @property
    def side(self) -> float:
        return self._side
    
    def area(self) -> float:
        return self._side ** 2


# Both work correctly when used as Shape
shapes: list[Shape] = [Rectangle(5, 4), Square(5)]
for shape in shapes:
    print(f"{type(shape).__name__} area: {shape.area()}")

## 4.3 Another LSP Example: Payment Processing

In [ ]:
# ✅ GOOD: Payment processors that respect LSP

class PaymentProcessor(ABC):
    """Contract: All processors must process payments and return success/failure."""
    
    @abstractmethod
    def process(self, amount: float) -> bool:
        """Process payment, return True if successful."""
        pass


class CreditCardProcessor(PaymentProcessor):
    def process(self, amount: float) -> bool:
        print(f"Processing ${amount} via credit card...")
        return True  # Simulated success


class PayPalProcessor(PaymentProcessor):
    def process(self, amount: float) -> bool:
        print(f"Processing ${amount} via PayPal...")
        return True


class BitcoinProcessor(PaymentProcessor):
    def process(self, amount: float) -> bool:
        print(f"Processing ${amount} via Bitcoin...")
        return True


def checkout(processor: PaymentProcessor, amount: float) -> str:
    """Works with ANY payment processor — LSP compliant."""
    if processor.process(amount):
        return "Payment successful!"
    return "Payment failed."


# All processors can substitute for PaymentProcessor
processors = [CreditCardProcessor(), PayPalProcessor(), BitcoinProcessor()]
for proc in processors:
    print(checkout(proc, 99.99))
    print()

---

# 5. Interface Segregation Principle (ISP)

> "Clients should not be forced to depend on interfaces they don't use."

Instead of one large interface with many methods, create smaller, focused interfaces. Classes should only implement what they actually need.

## 5.1 Violation: Fat Interface

In [ ]:
# ❌ BAD: One fat interface forces all workers to implement everything

class WorkerBad(ABC):
    """Violates ISP: All workers must implement all methods."""
    
    @abstractmethod
    def work(self) -> None:
        pass
    
    @abstractmethod
    def eat(self) -> None:
        pass
    
    @abstractmethod
    def sleep(self) -> None:
        pass


class HumanWorkerBad(WorkerBad):
    """Human can work, eat, and sleep — fine."""
    
    def work(self) -> None:
        print("Human working...")
    
    def eat(self) -> None:
        print("Human eating...")
    
    def sleep(self) -> None:
        print("Human sleeping...")


class RobotWorkerBad(WorkerBad):
    """Robot can only work — forced to implement useless methods!"""
    
    def work(self) -> None:
        print("Robot working...")
    
    def eat(self) -> None:
        pass  # Robots don't eat! Forced to implement useless method
    
    def sleep(self) -> None:
        pass  # Robots don't sleep! Another useless implementation


robot = RobotWorkerBad()
robot.work()
robot.eat()  # Does nothing — misleading interface

## 5.2 Correct Approach: Segregated Interfaces

In [ ]:
# ✅ GOOD: Separate interfaces for different capabilities

class Workable(ABC):
    """Interface for entities that can work."""
    
    @abstractmethod
    def work(self) -> None:
        pass


class Eatable(ABC):
    """Interface for entities that can eat."""
    
    @abstractmethod
    def eat(self) -> None:
        pass


class Sleepable(ABC):
    """Interface for entities that can sleep."""
    
    @abstractmethod
    def sleep(self) -> None:
        pass


class Human(Workable, Eatable, Sleepable):
    """Humans can work, eat, and sleep."""
    
    def work(self) -> None:
        print("Human working...")
    
    def eat(self) -> None:
        print("Human eating lunch...")
    
    def sleep(self) -> None:
        print("Human sleeping...")


class Robot(Workable):
    """Robots only work — they don't implement interfaces they don't need."""
    
    def work(self) -> None:
        print("Robot working 24/7...")


# Usage — polymorphism with specific interfaces
workers: list[Workable] = [Human(), Robot()]
for worker in workers:
    worker.work()  # Both can work

print()

# Only humans eat
human = Human()
human.eat()

## 5.3 Another ISP Example: Document Operations

In [ ]:
# ✅ GOOD: Segregated interfaces for document operations

class Readable(ABC):
    @abstractmethod
    def read(self) -> str:
        pass


class Writable(ABC):
    @abstractmethod
    def write(self, content: str) -> None:
        pass


class Deletable(ABC):
    @abstractmethod
    def delete(self) -> None:
        pass


class EditableDocument(Readable, Writable, Deletable):
    """Full-featured document with all operations."""
    
    def __init__(self, content: str = ""):
        self._content = content
    
    def read(self) -> str:
        return self._content
    
    def write(self, content: str) -> None:
        self._content = content
    
    def delete(self) -> None:
        self._content = ""


class ReadOnlyDocument(Readable):
    """Read-only document — only implements Readable."""
    
    def __init__(self, content: str):
        self._content = content
    
    def read(self) -> str:
        return self._content


def display_document(doc: Readable) -> None:
    """Works with anything Readable."""
    print(f"Content: {doc.read()}")


# Both can be read
editable = EditableDocument("Editable content")
readonly = ReadOnlyDocument("Protected content")

display_document(editable)
display_document(readonly)

---

# 6. Dependency Inversion Principle (DIP)

> "Depend on abstractions, not concretions."

High-level modules should not depend on low-level modules. Both should depend on abstractions.

## 6.1 Violation: Hard Dependencies

In [ ]:
# ❌ BAD: High-level class depends on low-level implementation

class MySQLDatabase:
    """Low-level: specific database implementation."""
    
    def save(self, data: dict) -> None:
        print(f"Saving to MySQL: {data}")


class ReportGeneratorBad:
    """Violates DIP: Hard dependency on MySQLDatabase."""
    
    def __init__(self):
        self.database = MySQLDatabase()  # Hard dependency!
    
    def generate(self, data: list) -> None:
        report = {"report": data, "type": "summary"}
        self.database.save(report)


# Problem: Can't easily switch databases or test without MySQL
gen = ReportGeneratorBad()
gen.generate([1, 2, 3])

## 6.2 Correct Approach: Depend on Abstractions

In [ ]:
# ✅ GOOD: Depend on abstraction (interface)

class Database(ABC):
    """Abstraction: Any database must implement save."""
    
    @abstractmethod
    def save(self, data: dict) -> None:
        pass


class MySQLDatabase(Database):
    def save(self, data: dict) -> None:
        print(f"Saving to MySQL: {data}")


class PostgreSQLDatabase(Database):
    def save(self, data: dict) -> None:
        print(f"Saving to PostgreSQL: {data}")


class InMemoryDatabase(Database):
    """For testing — no actual database needed."""
    
    def __init__(self):
        self.data: list[dict] = []
    
    def save(self, data: dict) -> None:
        self.data.append(data)
        print(f"Saved to memory: {data}")


class ReportGenerator:
    """Follows DIP: Depends on Database abstraction."""
    
    def __init__(self, database: Database):  # Inject dependency
        self.database = database
    
    def generate(self, data: list) -> dict:
        report = {"report": data, "type": "summary"}
        self.database.save(report)
        return report


# Easy to swap implementations
print("=== Production (MySQL) ===")
mysql_gen = ReportGenerator(MySQLDatabase())
mysql_gen.generate([1, 2, 3])

print("\n=== Production (PostgreSQL) ===")
pg_gen = ReportGenerator(PostgreSQLDatabase())
pg_gen.generate([1, 2, 3])

print("\n=== Testing (In-Memory) ===")
test_db = InMemoryDatabase()
test_gen = ReportGenerator(test_db)
test_gen.generate([1, 2, 3])
print(f"Stored data: {test_db.data}")

## 6.3 DIP Enables Testability

In [ ]:
# Testing with DIP is straightforward

def test_report_generator():
    """Test using mock database — no real database needed."""
    # Arrange
    mock_db = InMemoryDatabase()
    generator = ReportGenerator(mock_db)
    
    # Act
    result = generator.generate(["item1", "item2"])
    
    # Assert
    assert len(mock_db.data) == 1
    assert mock_db.data[0]["report"] == ["item1", "item2"]
    assert result["type"] == "summary"
    
    print("✅ All tests passed!")


test_report_generator()

---

# 7. Exercises

| Exercise | Topics | Difficulty |
|----------|--------|-----------|
| 1 | SRP Refactoring | Basic |
| 2 | OCP Extension | Intermediate |
| 3 | Complete SOLID | Advanced |

## Exercise 1: Refactor for SRP

The following `OrderProcessor` class violates SRP. Refactor it into separate classes:

- `Order` — data class for order information
- `OrderValidator` — validates orders
- `PaymentProcessor` — handles payments
- `EmailNotifier` — sends notifications

In [ ]:
# Exercise 1: Refactor for SRP - Original Code

class OrderProcessorBad:
    """Violates SRP: Does too many things."""
    
    def __init__(self, order_id: int, customer_email: str, items: list[str], total: float):
        self.order_id = order_id
        self.customer_email = customer_email
        self.items = items
        self.total = total
    
    def validate(self) -> bool:
        return len(self.items) > 0 and self.total > 0
    
    def process_payment(self) -> bool:
        print(f"Processing ${self.total} payment...")
        return True
    
    def send_confirmation(self) -> None:
        print(f"Sending confirmation to {self.customer_email}...")


# TODO: Refactor into separate classes following SRP

# class Order:
#     ...

# class OrderValidator:
#     ...

# class PaymentProcessor:
#     ...

# class EmailNotifier:
#     ...

## Exercise 2: Apply OCP with New Exporters

Create an extensible export system:

1. Create an `Exporter` ABC with `export(data: dict) -> str` method
2. Implement `JSONExporter`, `XMLExporter`, `CSVExporter`
3. Create an `ExportManager` that works with any exporter
4. Add a new `YAMLExporter` without modifying existing classes

In [ ]:
# Exercise 2: Apply OCP - Starter Code

class Exporter(ABC):
    """Abstract exporter — new formats extend this."""
    
    @abstractmethod
    def export(self, data: dict) -> str:
        pass


class JSONExporter(Exporter):
    # TODO: Implement JSON export
    pass


class XMLExporter(Exporter):
    # TODO: Implement XML export (simple string format OK)
    pass


class CSVExporter(Exporter):
    # TODO: Implement CSV export (simple string format OK)
    pass


class ExportManager:
    """Works with any exporter — never needs modification."""
    
    def __init__(self, exporter: Exporter):
        # TODO: Store exporter
        pass
    
    def export(self, data: dict) -> str:
        # TODO: Delegate to exporter
        pass


# === Demo (uncomment after implementation) ===
# data = {"name": "Report", "value": 42}
# 
# json_manager = ExportManager(JSONExporter())
# print(f"JSON: {json_manager.export(data)}")
# 
# xml_manager = ExportManager(XMLExporter())
# print(f"XML: {xml_manager.export(data)}")

## Exercise 3: Complete SOLID System

Design a notification system that follows all SOLID principles:

**Requirements:**
1. **SRP**: Separate classes for Message, Sender, Logger
2. **OCP**: Easy to add new sender types (Email, SMS, Push)
3. **LSP**: All senders can substitute for base Sender
4. **ISP**: Separate interfaces for Sendable, Loggable, Retryable
5. **DIP**: NotificationService depends on abstractions

In [ ]:
# Exercise 3: Complete SOLID System - Starter Code

# ISP: Separate interfaces
class Sendable(ABC):
    @abstractmethod
    def send(self, recipient: str, message: str) -> bool:
        pass


class Loggable(ABC):
    @abstractmethod
    def log(self, message: str) -> None:
        pass


# SRP: Message is just data
class Message:
    def __init__(self, recipient: str, content: str):
        self.recipient = recipient
        self.content = content


# OCP/LSP: Sender implementations
class EmailSender(Sendable):
    # TODO: Implement email sending
    pass


class SMSSender(Sendable):
    # TODO: Implement SMS sending
    pass


# DIP: Service depends on abstractions
class NotificationService:
    def __init__(self, sender: Sendable, logger: Loggable | None = None):
        # TODO: Store dependencies
        pass
    
    def notify(self, message: Message) -> bool:
        # TODO: Send message and optionally log
        pass


# === Demo (uncomment after implementation) ===
# email_service = NotificationService(EmailSender())
# msg = Message("user@example.com", "Hello!")
# email_service.notify(msg)

---

## 📝 Summary

This unit covered the five SOLID principles for object-oriented design:

**S — Single Responsibility Principle**
- Each class should have one reason to change
- Separate concerns into focused classes
- Benefits: easier testing, maintenance, and reuse

**O — Open/Closed Principle**
- Open for extension, closed for modification
- Add new behavior through new classes, not by changing existing code
- Use inheritance and composition for extensibility

**L — Liskov Substitution Principle**
- Subclasses must be substitutable for base classes
- Avoid inheritance that changes expected behavior
- Use proper abstractions instead of forcing inheritance

**I — Interface Segregation Principle**
- Many specific interfaces over one general interface
- Clients shouldn't depend on methods they don't use
- Split large ABCs into focused, single-purpose interfaces

**D — Dependency Inversion Principle**
- Depend on abstractions, not concrete implementations
- Use dependency injection for flexibility
- Enables easy testing and swapping implementations

---

**Next:** Unit 8c — Design Patterns